In [ ]:
import time
from tqdm import tqdm_notebook as tqdm

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
sns.set_style("whitegrid")
rcParams.update({'figure.autolayout': True})

In [ ]:
df = pd.read_json("./clean_data.dat")
df.head()

In [ ]:
df = df.loc[df.loc[:, "Kategorija pokazatelja"].notna()]
df.loc[:, "SR.VR."] = df.loc[:, "SR.VR."].apply(lambda x: "".join(x.split(",")) if len(x.split()) <=2 else np.nan).apply(lambda x: np.nan if x == "" else x).apply(float)
df = df.loc[df.loc[:, "SR.VR."].notna()]

df.loc[:, "br.an."] = df.loc[:, "br.an."].astype(float)

In [ ]:
df.loc[:, "ukupno"] = df.loc[:, "br.an."] * df.loc[:, "SR.VR."]

In [ ]:
df.info()

In [ ]:
vodniTipovi = [item for item in list(df.loc[:, "Vodi tip"].unique()) if item != ""]

In [ ]:
kategorijePokazatelja = [item for item in list(df.loc[:, "Kategorija pokazatelja"].unique()) if item is not np.nan]

In [ ]:
pokazateljiPoKategorijama = {kategorija : [item for item in list(df.loc[df.loc[:, "Kategorija pokazatelja"] == kategorija, "Pokazatelj"].unique()) if item != ""] for kategorija in kategorijePokazatelja}

In [ ]:
frame = df.copy()

Treba pivotat tablicu.<br>
Svaki pokazatalje će biti stupac, redak će predstavljati srednje vrijednosti identificirane (indeksirane) po lokaciji.

In [ ]:
def singlePerm(twoValFrame, nPerm):
    permArray = np.zeros(nPerm)
    for i in range(nPerm):
        col1 = list(twoValFrame.columns)[1]
        twoValFrame.loc[:, col1] = np.random.permutation(twoValFrame.loc[:, col1].values)
        permArray[i] = twoValFrame.corr().values[0, 1]
    return permArray.copy()
    

def corrPerm(dFrame, nPerm = 5_000):
    corrFrame = pd.DataFrame()
    cols = list(dFrame.columns)
    for i in range(len(cols)):
        for j in range(i + 1, len(cols)):
            corrFrame.loc[:, "corr({} ,{})".format(cols[i], cols[j])] = singlePerm(
                dFrame.loc[:, [cols[i], cols[j]]], nPerm)
    return corrFrame

In [ ]:
alpha = 0.05
for tip in tqdm([None] + vodniTipovi):
    if tip is not None:
        tmpFrame = frame.loc[frame.loc[:, "Vodi tip"] == tip]
    else:
        tmpFrame = frame
        tip = "Svi"
    
    ukupnoFrame = tmpFrame.pivot_table(values = "ukupno",
                                    columns="Pokazatelj", index = "Naziv postaje", aggfunc=np.sum)
    countFrame = tmpFrame.pivot_table(values = "br.an.",
                                   columns="Pokazatelj", index = "Naziv postaje", aggfunc=np.sum)
    meanFrame = ukupnoFrame / countFrame
    
    fig, axes = plt.subplots(2, 1, figsize = (30, 32))
    fig2, axes2 = plt.subplots(2, 1, figsize = (30, 32))
    fig3, ax3 = plt.subplots(figsize = (30, 16))
    dropFig, dropAx = plt.subplots(figsize = (30, 16))
    
    tmpCorr =  meanFrame.corr().applymap(lambda x: "{:.2f}".format(x)).astype(float)
    pVals = tmpCorr.copy()
    signif = tmpCorr.copy()
    features = list(tmpCorr.columns)
    for i in tqdm(range(len(features)), leave = False):
        for j in tqdm(range(i + 1, len(features)), leave = False):
            corrVal = tmpCorr.loc[features[i], features[j]]
            permSamples = corrPerm(meanFrame.loc[:, [features[i], features[j]]])
            if np.isnan(corrVal):
                pVal = 1.0
            elif corrVal >= 0:
                pVal = (permSamples > corrVal).mean()[0]
            else:
                pVal = (permSamples < corrVal).mean()[0]
            pVals.loc[features[i], features[j]] = pVal
            pVals.loc[features[j], features[i]] = pVal
            signif.loc[features[i], features[j]] = float(pVal < alpha)
            signif.loc[features[j], features[i]] = float(pVal < alpha)
            
    sns.heatmap(tmpCorr, cmap = "inferno", ax = axes[0], annot = True)
    sns.heatmap(pVals, cmap = "inferno", ax = axes[1], annot = True)
    
    sns.heatmap(tmpCorr, cmap = "inferno", ax = axes2[0], annot = True)
    sns.heatmap(signif, cmap = "inferno", ax = axes2[1], annot = True)
    
    tmpCorr.fillna(0, inplace = True)
    signif.fillna(0, inplace = True)
    
    sns.heatmap(tmpCorr, cmap = "inferno", ax = dropAx, annot = True)
    sns.heatmap(signif, cmap = "inferno", ax = ax3, annot = True)
    
    
    
    
    def corrTrans(value):
        if value == "250.0":
            return " "
        else:
            return value
    
    for t1, t2 in zip(ax3.texts, dropAx.texts):
        t1.set_text(corrTrans(t2.get_text()))
    
    axes[0].set_title("Vodni tip: {}".format(tip))
    axes[1].set_title("p-vrijednost")
    fig.savefig("./pics/corrPermTest/corrPlot_{}.pdf".format(tip))
    plt.show(fig)
    
    axes2[0].set_title("Vodni tip: {}".format(tip))
    axes2[1].set_title("značajnost")
    fig2.savefig("./pics/corrPermTest/corrPlotsignif_{}.pdf".format(tip))
    plt.show(fig2)
    
    ax3.set_title("Vodni tip: {}".format(tip))
    fig3.savefig("./pics/corrMerge/corrPlot_{}.pdf".format(tip))
    plt.show(fig3)
    plt.show(dropFig)